In [9]:
import dotenv
import os

from genai.credentials import Credentials
from genai.extensions.langchain import LangChainInterface
from genai.schemas import GenerateParams
from genai.model import Model

# from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
# from ibm_watson_machine_learning.foundation_models import Model
# from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
# from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM

In [34]:
dotenv.load_dotenv()

api_key = os.getenv("GENAI_KEY", None)
api_endpoint = os.getenv("GENAI_API", None)

creds = Credentials(api_key,api_endpoint)

params = GenerateParams(
    decoding_method="greedy",
    max_new_tokens=1000,
    min_new_tokens=15,
    # stop_sequences=['\n\n']
)

In [46]:
# model = Model(ModelTypes.LLAMA_2_70B_CHAT,creds,params,project_id)
# model = Model(model="bigcode/starcoder",credentials=creds, params=params)

# llm = LangChainInterface(model="bigcode/starcoder", credentials=creds, params=params)
llm = Model(model="meta-llama/llama-2-70b-chat",credentials=creds, params=params)

In [26]:
javacode = open("/Users/yingkitw/Desktop/watsonx-demo/code_assistant/AuthTurntableService.java").read()
# print(javacode)

In [102]:
import javalang

def split_java_code(java_code):
    import_part = ""
    class_part = ""
    method_parts = []
    closing_part = ""

    tree = javalang.parse.parse(java_code)

    for path, node in tree:
        if isinstance(node, javalang.tree.Import):
            import_part += str(node) + "\n"
        elif isinstance(node, javalang.tree.ClassDeclaration):
            class_part += get_node_with_comments(node, java_code) + "\n"
        elif isinstance(node, javalang.tree.MethodDeclaration):
            method_parts.append(get_node_with_comments(node, java_code) + "\n")
        else:
            closing_part += get_node_with_comments(node, java_code) + "\n"

    alist = [import_part.strip()]
    alist += [class_part.strip()]
    alist += method_parts
    alist += [closing_part.strip()]

    return alist


def get_node_with_comments(node, java_code):
    start_line = node.position.line - 1 if hasattr(node, 'position') else 0
    end_line = node.position.line - 1 if hasattr(node, 'position') else len(java_code.splitlines())
    for path, current_node in javalang.parse.parse(java_code):
        if current_node == node and hasattr(current_node, 'position') and current_node.position:
            start_line = current_node.position.line - 1
            end_line = current_node.position.line - 1
        elif hasattr(current_node, 'position') and current_node.position and current_node.position.line > node.position.line and current_node.position.line < end_line:
            end_line = current_node.position.line - 1
    lines = java_code.splitlines()
    node_lines = lines[start_line:end_line+1]
    return "\n".join(node_lines)

parts = split_java_code(javacode)
for part in parts:
    print("part: "+part)

AttributeError: 'NoneType' object has no attribute 'line'

In [128]:
import javalang.tokenizer as tokenizer

java_code = """public class MyClass {
public void myMethod() {
System.out.println("Hello, world!");
}
}
"""

tree = javalang.parse.parse(java_code)

for path, node in tree:
    if isinstance(node, javalang.tree.MethodDeclaration):
        print(f"{node.position.line},{node.position.column}")
        for _, child in node:
            print(child.position)
            print(java_code.splitlines()[child.position.line-1])
        # method_content = java_code[node.position:]
        # print(method_content)

2,8
Position(line=2, column=8)
public void myMethod() {
Position(line=3, column=1)
System.out.println("Hello, world!");
Position(line=3, column=1)
System.out.println("Hello, world!");
Position(line=3, column=20)
System.out.println("Hello, world!");


In [152]:
java_code = """
// This is an import statement
import java.util.*;

/* This is a multi-line comment
   that spans multiple lines */
import java.io.*;

/**
 * This is a Java class
 */
public class MyClass {
    // This is a variable
    private int myVariable;

    /**
     * This is a Java method
     */
    public void myMethod() {
        // Method implementation
    }
"""

prompts = [f"""[INST]indicate me as ^ the position of the end of methods with rationale.
    java code:`{java_code}`
    [/INST]indicate the end with ^ and line, column:"""]

params = GenerateParams(
    decoding_method="greedy",
    max_new_tokens=500,
    min_new_tokens=1,
)
llm = Model(model="meta-llama/llama-2-70b-chat",credentials=creds, params=params)

for response in llm.generate(prompts):
    print(response.generated_text)
    # print(java_code[0:100])
    



1. import statement: ^1, 1-2
2. multi-line comment: ^3, 4-6
3. variable: ^7, 8-9
4. Java method: ^10, 11-14
5. Method implementation: ^15, 16-17

Explanation:

1. The import statement ends at the closing curly brace `}` on line 2, column 2.
2. The multi-line comment ends at the closing `*/` on line 6, column 2.
3. The variable `myVariable` ends at the semicolon `;` on line 9, column 1.
4. The Java method `myMethod()` ends at the closing curly brace `}` on line 14, column 2.
5. The method implementation (i.e., the code inside the method) ends at the closing curly brace `}` on line 17, column 2.

Note that the line and column numbers are based on the provided Java code and may vary depending on the actual file and editor used.


In [85]:
def buildprompt(java):
    # - show the code on imports
# - show the class definition
# - show the class variable
# - show the class constructor

    prompt = f"""[INST]
read the java code in backquoted.
analysis if any bad coding practice
java code:`{java}`
[/INST]
analysis:
"""
    return prompt

prompts = []
for part in parts:
    print(len(part))
    print(part)
    prompts += [buildprompt(parts)]

# for response in llm.generate(prompts):
#     print(response.generated_text)

1579
import cn.hutool.core.util.ObjectUtil;
import cn.hutool.core.util.StrUtil;
import com.alibaba.fastjson.JSON;
import com.hongqi.market.api.CommonResult;
import com.hongqi.market.base.constant.Constant;
import com.hongqi.market.config.CmnfawConfig;
import com.hongqi.market.config.JwtConfig;
import com.hongqi.market.dto.RequestParameters;
import com.hongqi.market.dto.request.activity.template.turntable.lottery.LoginForm;
import com.hongqi.market.dto.request.activity.template.turntable.lottery.SendCodeForm;
import com.hongqi.market.dto.response.activity.template.turntable.lottery.HomeFrom;
import com.hongqi.market.dto.response.activity.template.turntable.lottery.LotteryRuleForm;
import com.hongqi.market.dto.vo.activity.template.turntable.lottery.TokenVo;
import com.hongqi.market.feign.TripartiteService;
import com.hongqi.market.mapper.NovaTurntableExcelImportMapper;
import com.hongqi.market.sms.ApiResult;
import com.hongqi.market.sms.SmsUtils;
import com.hongqi.market.utils.*;
import 

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceInstructEmbeddings
db = Chroma.from_documents(texts, HuggingFaceInstructEmbeddings())
retriever = db.as_retriever(
    search_type="mmr", # Also test "similarity"
    search_kwargs={"k": 8},
)

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [ ]:
question = "Generate a SQL to create Table with 2 column: Name, Id"
result = qa(question)
result['answer']

In [ ]:
questions = [
    "What is the class hierarchy?",
    "What classes are derived from the Chain class?",
    "What one improvement do you propose in code in relation to the class herarchy for the Chain class?",
]

for question in questions:
    result = qa(question)
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")